# Perspective Streaming Data Example
In this example, we will use [csp](https://github.com/point72/csp) and [superstore](https://github.com/timkpaine/superstore) to build a streaming data graph, and pump data into `perspective` widgets.

`superstore` is a library with some simulated data sources, and `csp` is a stream processing library. The details of `csp` are not super important for the purpose of this talk, but here we construct a streaming DAG of data which will feed `perspective` at different intervals

In [1]:
import csp
import pandas as pd
from datetime import timedelta
from ipywidgets import HBox, VBox
from perspective import PerspectiveWidget
from csp_nodes import *

In [2]:
m = machines()
pd.DataFrame(m)

,machine_id,kind,cores,region,zone
0,cad15a8e3fe9,worker,64,na,A
1,512d6194ca5c,edge,8,eu,D
2,375ddf95227b,worker,64,ap,C
3,d6187389714e,edge,8,eu,D
4,6c29573bd069,edge,4,ap,A
...,...,...,...,...,...
95,3c0fb26783d3,core,16,eu,C
96,96d46d4ccc5a,core,16,eu,A
97,667a77c79630,edge,4,na,A
98,509488895a9f,worker,64,eu,A


In [3]:
u = usage(m[4])
u = usage(u)
u

{'machine_id': '6c29573bd069',
 'kind': 'edge',
 'cores': 4,
 'region': 'ap',
 'zone': 'A',
 'cpu': 36.09,
 'mem': 53.31,
 'free': 46.69,
 'network': 65.46,
 'disk': 71.68}

In [4]:
status(u)

{'machine_id': '6c29573bd069',
 'kind': 'edge',
 'cores': 4,
 'region': 'ap',
 'zone': 'A',
 'cpu': 36.09,
 'mem': 53.31,
 'free': 46.69,
 'network': 65.46,
 'disk': 71.68,
 'last_update': datetime.datetime(2024, 5, 23, 13, 0, 47, 112740),
 'status': 'active'}

In [9]:
jobs(u)

In [10]:
@csp.graph
def main_graph(
    machines_widget: PerspectiveWidget,
    usage_widget: PerspectiveWidget,
    status_widget: PerspectiveWidget,
    jobs_widget: PerspectiveWidget,
):
    # A randomly generated list of "machines"
    all_machines = machines()

    # construct three ticking nodes for usage, status, and jobs
    usage = machine_usage(all_machines, timedelta(seconds=1))
    status = machine_status(usage, timedelta(seconds=5))
    jobs = machine_jobs(all_machines, timedelta(seconds=5))

    # push all of our data to 4 separate perspective tables
    push_to_perspective(csp.const(all_machines), machines_widget)
    push_to_perspective(usage, usage_widget)
    push_to_perspective(status, status_widget)
    push_to_perspective(jobs, jobs_widget)

In [11]:
# construct 4 separate perspective widgets. Each will have its own table internally
machines_widget = PerspectiveWidget(MACHINE_SCHEMA, index="machine_id", settings=False)
usage_widget = PerspectiveWidget(USAGE_SCHEMA, index="machine_id", settings=False)
status_widget = PerspectiveWidget(STATUS_SCHEMA, index="machine_id", sort=[["last_update", "desc"]], settings=False)
jobs_widget = PerspectiveWidget(JOBS_SCHEMA, sort=[["start_time", "desc"]], settings=False)

In [12]:
# a little bit of layout with ipywidgets
VBox(children=[
    HBox(children=[machines_widget, usage_widget]),
    HBox(children=[status_widget, jobs_widget]),
])

In [13]:
csp.run_on_thread(main_graph, machines_widget, usage_widget, status_widget, jobs_widget, realtime=True, daemon=True)

In [14]:
status_widget.plugin = "X Bar"
status_widget.group_by = ["status"]
status_widget.columns = ["machine_id"]
status_widget.aggregates = {"status": "last"}